# Example Convolutional Neural Network for mnist-data

Needed Imports

In [0]:
import numpy as np
import keras
import keras.layers as layers
import matplotlib.pyplot as plt
from keras.datasets import mnist

Load data, split training set in training and validation set, reshape sets

In [0]:
(traini, trainl), (testi, testl) = mnist.load_data()

fac = 1.001 * 255
testi = testi / fac + 0.001
traini = traini / fac + 0.001

testi = testi - np.mean(testi, axis=0)
traini = traini - np.mean(traini, axis=0)

train_size = len(trainl)
val_size = int(train_size / 5)
train_size = train_size - val_size
vali = traini[train_size:]
vall = trainl[train_size:]
traini = traini[:train_size]
trainl = trainl[:train_size]

traini = traini.reshape(len(traini), 28, 28, 1)
vali = vali.reshape(len(vali), 28, 28, 1)
testi = testi.reshape(len(testi), 28, 28, 1)

trainl = keras.utils.to_categorical(trainl, num_classes=None)
vall = keras.utils.to_categorical(vall, num_classes=None)
testl = keras.utils.to_categorical(testl, num_classes=None)

Some hyperparameters

In [0]:
epochs = 48
batch_size = 64
num_classes = 10

reg = 3e-3

Define Model:

      Input => Conv => Relu => Conv => Relu => Pool => Conv => Relu => Conv => Relu => FC =>  Softmax => Output

Compile Model: adam-optimizer, softmax crossentropy

In [0]:
model = keras.Sequential()

model.add(
    layers.Conv2D(
        input_shape=traini.shape[1:],
        activation='relu',
        filters=8,
        kernel_size=3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    )
)

model.add(
    layers.Conv2D(
        activation='relu',
        filters=8,
        kernel_size=3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    )
)

model.add(
    layers.MaxPooling2D(
        pool_size=2
    )
)

model.add(
    layers.Conv2D(
        activation='relu',
        filters=8,
        kernel_size=3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    )
)

model.add(
    layers.Conv2D(
        activation='relu',
        filters=8,
        kernel_size=3,
        padding='same',
        kernel_regularizer=keras.regularizers.l2(reg)
    )
)

model.add(
    layers.Flatten()
)

model.add(
    layers.Dense(
        num_classes,
        activation='softmax',
        kernel_regularizer=keras.regularizers.l2(reg)
    )
)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

How good ist randomly initialized model?

In [0]:
# Score untrained model.
scores_untrained = model.evaluate(testi, testl, verbose=1)

Training!

In [0]:
history = model.fit(
    traini, trainl,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(vali, vall),
    shuffle=True
)

How good is trained model?

In [0]:
# Score trained model.
scores = model.evaluate(testi, testl, verbose=1)

print('\nTest loss untrained:', scores_untrained[0])
print('Test accuracy untrained:', scores_untrained[1])
print('\nTest loss:', scores[0])
print('Test accuracy:', scores[1])

model.summary()

Grap random test image, show it inline and predict with trained model

In [0]:
index = np.random.randint(0,testl.shape[0])
img = testi[index].reshape((28, 28))
plt.imshow(img, cmap="Greys")
plt.show()

prediction = model.predict(testi[index].reshape(1, 28, 28, 1))
print('prediction:', np.where(prediction[0].round(1) == 1.0))

Later maybe download model settings and restore them

In [0]:
json_string = model.to_json()
with open('mnist_model.json', 'w') as file:
  file.write(json_string + '\n')
model.save_weights('mnist_weights.hdf5')